In [2]:
import tensorflow as tf

2023-12-13 17:13:00.965159: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-13 17:13:01.092164: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 17:13:01.092272: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 17:13:01.100672: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 17:13:01.135742: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-13 17:13:01.136754: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
import tensorflow_datasets as tfds
import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tensorflow_examples.models.pix2pix import pix2pix
import matplotlib.pyplot as plt

In [4]:
dataset_directory = 'Image Dataset'  
batch_size = 32  
image_size = (256,256)  
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_directory,
    shuffle=True,
    batch_size=batch_size,
    image_size=image_size
)
dataset_directory = 'Inverted Images Dataset'  
batch_size = 32
image_size = (256,256)  
rotated_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_directory,
    shuffle=True,
    batch_size=batch_size,
    image_size=image_size
)
AUTOTUNE = tf.data.AUTOTUNE
#dataset = dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
#rotated_dataset = dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

Found 498 files belonging to 1 classes.
Found 498 files belonging to 1 classes.


In [5]:
def normalize(image):
    img = tf.cast(img, dtype=tf.float32)
    return (img / 127.5) - 1.0
def preprocess_image(image):
    image = normalize(image)
    return image

In [6]:
BUFFER_SIZE = 64
BATCH_SIZE = 32
IMG_WIDTH = 256
IMG_HEIGHT = 256
OUTPUT_CHANNELS = 3

In [7]:
import tensorflow as tf

def filter_batch_size(batch, batch_size=32):
    return tf.shape(batch)[0] == batch_size
filtered_dataset = dataset.filter(lambda x, y: filter_batch_size(x, batch_size=32))
filtered_rotated_dataset = rotated_dataset.filter(lambda x, y: filter_batch_size(x, batch_size=32))
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

In [8]:
example = next(iter(dataset))

In [9]:
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32)
    return image, label

# Apply the preprocess function to the datasets
dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
rotated_dataset = rotated_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Batch and prefetch the datasets with drop_remainder=True
#dataset = dataset.batch(BATCH_SIZE, drop_remainder=False).prefetch(tf.data.AUTOTUNE)
#rotated_dataset = rotated_dataset.batch(BATCH_SIZE, drop_remainder=False).prefetch(tf.data.AUTOTUNE)

# Define the models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

print(dataset)
# Process each batch in the dataset
for batch in dataset:
    upright_images = batch[0]  # This extracts the image tensor from the batch
    upright_images = tf.squeeze(upright_images)
    print("Shape after squeeze:", upright_images.shape)
    if upright_images.shape[-1] == 3:
        to_flipped = generator_g(upright_images)
    else:
        print("Incorrect image shape")

for batch in rotated_dataset:
    rotated_images = batch[0]
    rotated_images = tf.squeeze(rotated_images)
    print("Shape after squeeze:", rotated_images.shape)
    if rotated_images.shape[-1] == 3:
        to_upright = generator_f(rotated_images)
    else:
        print("Incorrect image shape")


<_ParallelMapDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (18, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (32, 256, 256, 3)
Shape after squeeze: (3

In [10]:
LAMBDA = 10
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real, generated):
    real_loss = loss_obj(tf.ones_like(real), real)
    generated_loss = loss_obj(tf.zeros_like(generated), generated)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss * 0.5
def generator_loss(generated):
    return loss_obj(tf.ones_like(generated), generated)
def calc_cycle_loss(real_image, cycled_image):
    loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
    return LAMBDA * loss1
def identity_loss(real_image, same_image):
    loss = tf.reduce_mean(tf.abs(real_image - same_image))
    return LAMBDA * 0.1 * loss
generator_g_optimizer = tf.keras.optimizers.legacy.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.legacy.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.legacy.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.legacy.Adam(2e-4, beta_1=0.5)

In [11]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

In [12]:
EPOCHS = 500
def generate_images(model, test_input):
    prediction = model(test_input[0])
    plt.figure(figsize=(12, 12))
    display_list = [test_input[0], prediction[0]]
    title = ['Input Image', 'Predicted Image']
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
        plt.show()

In [13]:
@tf.function
def train_step(real_x, real_y):
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
    with tf.GradientTape(persistent=True) as tape:
        # Generator G translates X -> Y
        # Generator F translates Y -> X.

        fake_y = generator_g(real_x, training=True)
        cycled_x = generator_f(fake_y, training=True)

        fake_x = generator_f(real_y, training=True)
        cycled_y = generator_g(fake_x, training=True)

        # same_x and same_y are used for identity loss.
        same_x = generator_f(real_x, training=True)
        same_y = generator_g(real_y, training=True)

        disc_real_x = discriminator_x(real_x, training=True)
        disc_real_y = discriminator_y(real_y, training=True)

        disc_fake_x = discriminator_x(fake_x, training=True)
        disc_fake_y = discriminator_y(fake_y, training=True)

        # calculate the loss
        gen_g_loss = generator_loss(disc_fake_y)
        gen_f_loss = generator_loss(disc_fake_x)

        total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)

        # Total generator loss = adversarial loss + cycle loss
        total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
        total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)

  # Calculate the gradients for generator and discriminator
    generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
    generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                        generator_f.trainable_variables)

    discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
    discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)

  # Apply the gradients to the optimizer
    generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

    generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                            generator_f.trainable_variables))

    discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))

    discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

In [ ]:
for epoch in range(EPOCHS):
    start = time.time()
    n = 0
    for image_x, image_y in zip(dataset, rotated_dataset):
        # Assuming image_x and image_y are tuples of (image, label), 
        # and you only need the image for training
        train_step(image_x[0], image_y[0])
        if n % 10 == 0:
            print('.', end='')
        n += 1
    clear_output(wait=True)
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    print('Time taken for epoch {} is {} sec\n'.format(epoch + 1, time.time()-start))


..

In [ ]:
for inp in dataset.take(5):
    generate_images(generator_f, inp)

In [ ]:
def generate_images(model, test_input):
    # Assuming test_input is a tuple where the first item is the input data
    input_data = test_input[0]
    predictions = model(input_data, training=False)

    # Check the dimensions of input_data
    if input_data.ndim == 4:
        # Expected shape for a batch of images: (batch_size, height, width, channels)
        batch_size = input_data.shape[0]
    else:
        raise ValueError(f"Unexpected shape of input_data: {input_data.shape}")

    for i in range(batch_size):
        plt.figure(figsize=(10, 10))

        # Extract the input image and the corresponding prediction
        input_image = input_data[i].numpy() if hasattr(input_data[i], 'numpy') else input_data[i]
        predicted_image = predictions[i].numpy() if hasattr(predictions[i], 'numpy') else predictions[i]

        # Check if the image is in the correct shape
        if input_image.ndim != 3 or input_image.shape[-1] != 3:
            raise ValueError(f"Input image shape {input_image.shape} is not valid for display")

        if predicted_image.ndim != 3 or predicted_image.shape[-1] != 3:
            raise ValueError(f"Predicted image shape {predicted_image.shape} is not valid for display")

        # Display the input and predicted images
        for j, image in enumerate([input_image, predicted_image]):
            plt.subplot(1, 2, j+1)
            plt.title(['Input Image', 'Predicted Image'][j])
            plt.imshow(image * 0.5 + 0.5)  # Normalize pixel values
            plt.axis('off')

        plt.show()
